In [18]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from ResNet import ResNet
import numpy as np


device = "mps" if torch.backends.mps.is_built() \
    else "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(3)

## __Initial training of ResNet-56 on CIFAR10__

#### NN

In [13]:
model=ResNet(num_classes=10,n=9).to(device)

#### Load dataset

In [14]:
# Define standard data transforms for CIFAR10
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010]
    ),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010]
    ),
])


In [15]:
# Load datasets
train_dataset=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=train_transform)
test_dataset=torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=test_transform)

#### Define basic params

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

# Training hyperparameters
num_epochs = 270
batch_size = 1000
initial_lr = 0.1

# Learning rate schedule: warmup for 15 epochs, then step down
def lr_lambda(epoch):
    # Linear warmup for first 15 epochs
    if epoch < 15:
        # Start at a small value, linearly go to 1.0 over 15 epochs
        return float(epoch + 1) / 15
    elif epoch < 90:
        return 1.0
    elif epoch < 180:
        return 0.1
    elif epoch < 240:
        return 0.01
    else:
        return 0.001

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)


In [22]:
pin_memory = device == "cuda"
train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=4, pin_memory=pin_memory)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=4, pin_memory=pin_memory)

In [23]:
train_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(output, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    avg_loss = total_loss / len(train_loader)
    train_acc = correct / total if total > 0 else 0.

    model.eval()
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            _, predicted = torch.max(output, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
        val_acc = correct_val / total_val if total_val > 0 else 0.

    train_losses.append(avg_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    scheduler.step()

    # Print every 10th epoch
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {avg_loss:.4f} Train Acc: {train_acc:.4f} Val Acc: {val_acc:.4f}")


: 

In [ ]:
# from datasets import Dataset
# Dataset.cleanup_cache_files

/Users/ronibendom/Documents/hirundo_assigmnment/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<function datasets.arrow_dataset.Dataset.cleanup_cache_files(self) -> int>

In [2]:
from datasets import load_dataset
ds = load_dataset("hirundo-io/Noisy-CIFAR-100")

Generating test split: 100%|██████████| 10000/10000 [00:00<00:00, 21153.72 examples/s]


In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['png', '__key__', '__url__'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['png', '__key__', '__url__'],
        num_rows: 10000
    })
})